In [ ]:
import pandas as pd
from google.colab import files
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import RandomForestClassifier
from joblib import dump, load
import unicodedata
import re
from collections import Counter
files.upload()
!chmod 600 ./kaggle.json
os.environ['KAGGLE_CONFIG_DIR'] = '.'

In [ ]:
sub = pd.read_csv('submission_example.csv')
sub

,ID,RENDIMIENTO_GLOBAL
0,550236,medio-bajo
1,98545,medio-bajo
2,499179,medio-alto
3,782980,alto
4,785185,medio-bajo
...,...,...
296781,496981,alto
296782,209415,medio-alto
296783,239074,medio-alto
296784,963852,medio-bajo


In [ ]:
!kaggle competitions download -competitions udea-ai4eng-20241
!unzip -o "udea-ai4eng-20241.zip"

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/kaggle/cli.py", line 63, in main
    out = args.func(**command_args)
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 1037, in competition_download_cli
    self.competition_download_files(competition, path, force,
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 1000, in competition_download_files
    url = response.retries.history[0].redirect_location.split('?')[0]
IndexError: tuple index out of range
unzip:  cannot find or open udea-ai4eng-20241.zip, udea-ai4eng-20241.zip.zip or udea-ai4eng-20241.zip.ZIP.


In [ ]:
data = pd.read_csv('train.csv')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
data.shape

(692500, 17)

In [ ]:
# Eliminar filas que contienen cualquier NaN
data.dropna(inplace=True)

In [ ]:
data.shape

(628896, 17)

In [ ]:
# Eliminar la columna 'ESTU_PRIVADO_LIBERTAD' y una de las columnas 'FAMI_TIENEINTERNET'
data = data.drop(['ESTU_PRIVADO_LIBERTAD', 'FAMI_TIENEINTERNET.1'], axis=1)


In [ ]:

if 'RENDIMIENTO_GLOBAL' not in data.columns:
    raise ValueError("La columna 'RENDIMIENTO_GLOBAL' no se encuentra en el DataFrame.")


NameError: name 'data' is not defined

In [ ]:
# Función para remover acentos, caracteres especiales y vocales
def remove_accents_and_special_characters(input_str):
    # Remover acentos
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    without_accents = "".join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Remover caracteres especiales
    without_special_chars = re.sub(r'[^\w\s]', '', without_accents)

    # Remover vocales (a, e, i, o, u) y sus mayúsculas correspondientes
    without_vowels = re.sub(r'[aeiouAEIOU]', '', without_special_chars)

    return without_vowels


In [ ]:
# Función para remover palabras específicas
def remove_specific_words(name):
    words_to_remove = ['PRFSNL', 'PRFSNL D', 'PRFSNL N', 'FRMCN PRFSNL N', 'FRMCN PRFSNL N', 'FRMCN PRFSNL', 'FRMCN', 'STDS']
    for word in words_to_remove:
        name = name.replace(word, '')
    return name.strip()

In [ ]:
# Función para agrupar las categorías
def group_categories(name):
    if 'TRBJ SCL' in name:
        return 'TRBJ SCL'
    if 'GSTRNM' in name:
        return 'GSTRNM'
    if 'PSCLG' in name:
        return 'PSCLG'
    if 'NFRMR' in name:
        return 'NFRMR'
    if 'CNTDR' in name:
        return 'CNTDR'
    if 'NSTRMNTCN' in name:
        return 'NFRMR'
    if 'NGNR' in name:
        return 'NGNR'
    if 'DMNSTRCN' in name:
        return 'DMNSTRCN'
    if 'LCNCTR' in name:
        return 'LCNCTR'
    if 'FNNZS' in name:
        return 'FNNZS'
    if 'QMC' in name:
        return 'QMC'
    if 'MTMTCS' in name:
        return 'MTMTCS'
    if 'VTRNR' in name:
        return 'VTRNR'
    if 'ZTCN' in name:
        return 'VTRNR'
    if 'CMNCCN' in name:
        return 'CMNCCN_Y_PRDSM'
    if 'PRDSM' in name:
        return 'CMNCCN_Y_PRDSM'
    if 'TLVSN' in name:
        return 'CMNCCN_Y_PRDSM'
    if 'DRCH' in name:
        return 'DRCH Y CNCS PLTCS'
    if 'PLTC' in name:
        return 'DRCH Y CNCS PLTCS'
    if 'CRMNL' in name:
        return 'DRCH Y CNCS PLTCS'
    if 'MRCD' in name:
        return 'MRCD Y PBLCDD'
    if 'MRKTNG' in name:
        return 'MRCD Y PBLCDD'
    if 'PBLCDD' in name:
        return 'MRCD Y PBLCDD'

    if 'MDCN' in name:
        return 'CNCS SLD'
    if 'PSCLG' in name:
        return 'CNCS SLD'
    if 'NFRMR' in name:
        return 'CNCS SLD'
    if 'DNTLG' in name:
        return 'CNCS SLD'
    if 'MCRBLG' in name:
        return 'CNCS SLD'
    if 'FRMC' in name:
        return 'CNCS SLD'
    if 'NSTRMNTCN QRRGC' in name:
      return 'CNCS SLD'
    if 'N PSCLG' in name:
      return 'CNCS SLD'
    # Agrupaciones para ciencias básicas
    if 'MTMTCS' in name:
        return 'CNCS BSCS'
    if 'QMC' in name:
        return 'CNCS BSCS'
    if 'GLG' in name:
        return 'CNCS BSCS'
    if 'FSC' in name:
        return 'CNCS BSCS'
    if 'MCRBLG' in name:
        return 'CNCS BSCS'


    if 'HSTR' in name:
        return 'HMNDDS'
    if 'FLLG' in name:
        return 'HMNDDS'
    if 'SPNL ' in name:
        return 'HMNDDS'
    if 'TRDCCN' in name:
        return 'HMNDDS'
    if 'FLSF' in name:
        return 'HMNDDS'
    if 'CLTRL' in name:
        return 'HMNDDS'
    if 'TRRTRL' in name:
        return 'HMNDDS'
    if 'BBLTCLG' in name:
        return 'HMNDDS'



    else:
        return 'OTROS'

In [ ]:
def agrupar_top_n(transformada_columna, n=10):
    # Contar la frecuencia de cada categoría
    conteo_categorias = transformada_columna.value_counts()

    # Identificar las top n categorías más frecuentes
    top_n_categorias = conteo_categorias.nlargest(n).index

    # Función para agrupar en "OTROS"
    def agrupar_en_otros(categoria):
        if categoria in top_n_categorias:
            return categoria
        else:
            return 'OTROS'

    # Aplicar la función de agrupación a la columna transformada
    columna_agrupada = transformada_columna.map(agrupar_en_otros)

    return columna_agrupada

In [ ]:
# Nueva función para unificar la educación de los padres
def unificar_educacion_padres(data):
    # Combinar las columnas de educación de los padres, manejando valores nulos
    data['FAMI_EDUCACION_PADRES_COMBINADA'] = data.apply(
        lambda row: '-'.join(sorted([str(row['FAMI_EDUCACIONPADRE']), str(row['FAMI_EDUCACIONMADRE'])])), axis=1
    )

    # Eliminar las columnas originales de educación de los padres
    data.drop(['FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE'], axis=1, inplace=True)

    return data

In [ ]:
def to_onehot(x):
    values = np.unique(x)
    r = np.r_[[np.argwhere(i==values)[0][0] for i in x]]
    return np.eye(len(values))[r].astype(int)

In [ ]:
def replace_columns_with_onehot(d, col):
    k = to_onehot(d[col].values)
    r = pd.DataFrame(k, columns=["%s_%d"%(col, i) for i in range(k.shape[1])], index=d.index).join(d)
    del(r[col])
    return r

In [ ]:
def clean_data(data):
    data = unificar_educacion_padres(data)
    data.drop(['ID', 'ESTU_PRIVADO_LIBERTAD', 'PERIODO', 'FAMI_TIENELAVADORA',  'FAMI_TIENEINTERNET.1'], axis=1, inplace=True)
    columna_programa_academico = data['ESTU_PRGM_ACADEMICO']
    columna_programa_academico = columna_programa_academico.map(remove_accents_and_special_characters)
    columna_programa_academico = columna_programa_academico.map(remove_specific_words)
    columna_programa_academico = columna_programa_academico.map(group_categories)
    data['ESTU_PRGM_ACADEMICO_CATEGORIA'] = columna_programa_academico
    data.drop(['ESTU_PRGM_ACADEMICO'], axis=1, inplace=True)
    data['FAMI_ESTRATOVIVIENDA_DUPLICATE'] = data['FAMI_ESTRATOVIVIENDA']
    data['ESTU_HORASSEMANATRABAJA_DUPLICATE'] = data['ESTU_HORASSEMANATRABAJA']
    imputer = SimpleImputer(strategy='most_frequent')
    data = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)
    data.dropna(inplace=True)
    data = pd.get_dummies(data, columns=['ESTU_PRGM_ACADEMICO_CATEGORIA',
                                         'ESTU_PRGM_DEPARTAMENTO',
                                         'ESTU_VALORMATRICULAUNIVERSIDAD',
                                         'ESTU_HORASSEMANATRABAJA',
                                         'ESTU_HORASSEMANATRABAJA_DUPLICATE',
                                         'FAMI_ESTRATOVIVIENDA',
                                         'FAMI_ESTRATOVIVIENDA_DUPLICATE',
                                         'FAMI_TIENEINTERNET',
                                         'FAMI_TIENEAUTOMOVIL',
                                         'ESTU_PAGOMATRICULAPROPIO',
                                         'FAMI_TIENECOMPUTADOR',
                                         'FAMI_EDUCACION_PADRES_COMBINADA',
                                         ])

    return data

In [ ]:
# Cargar los datos de entrenamiento
data_train = pd.read_csv('train.csv')

In [ ]:
data_train

,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,ESTU_PRGM_ACADEMICO_TOP_10,FAMI_ESTRATOVIVIENDA_DUPLICATE,ESTU_HORASSEMANATRABAJA_DUPLICATE
0,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,No,Si,Postgrado,medio-alto,NFRMR,Estrato 3,Menos de 10 horas
1,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,No,Si,Técnica o tecnológica incompleta,bajo,DRCH Y CNCS PLTCS,Estrato 3,0
2,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,No,No,Secundaria (Bachillerato) completa,bajo,MRCD Y PBLCDD,Estrato 3,Más de 30 horas
3,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,No,Si,Secundaria (Bachillerato) completa,alto,DMNSTRCN,Estrato 4,0
4,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,No,Si,Primaria completa,medio-bajo,PSCLG,Estrato 3,Entre 21 y 30 horas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,Si,Si,Secundaria (Bachillerato) incompleta,medio-alto,BLG,Estrato 2,Entre 11 y 20 horas
692496,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Primaria incompleta,Si,No,No,Si,Secundaria (Bachillerato) incompleta,bajo,PSCLG,Estrato 3,Más de 30 horas
692497,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,Si,Si,Secundaria (Bachillerato) incompleta,medio-bajo,DMNSTRCN,Estrato 3,Menos de 10 horas
692498,TOLIMA,Entre 2.5 millones y menos de 4 millones,Menos de 10 horas,Estrato 1,No,Primaria completa,No,No,Si,Si,Primaria completa,bajo,PSCLG,Estrato 1,Menos de 10 horas


In [ ]:
data_c = clean_data(data_train)

In [ ]:
data_c

,RENDIMIENTO_GLOBAL,ESTU_PRGM_ACADEMICO_TOP_10_BCTRLG Y LBRTR CLNC,ESTU_PRGM_ACADEMICO_TOP_10_BLG,ESTU_PRGM_ACADEMICO_TOP_10_CMNCCN_Y_PRDSM,ESTU_PRGM_ACADEMICO_TOP_10_CMRC NTRNCNL,ESTU_PRGM_ACADEMICO_TOP_10_CNCS MLTRS,ESTU_PRGM_ACADEMICO_TOP_10_CNM,ESTU_PRGM_ACADEMICO_TOP_10_CNTDR,ESTU_PRGM_ACADEMICO_TOP_10_DMNSTRCN,ESTU_PRGM_ACADEMICO_TOP_10_DNTLG,...,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta
0,medio-alto,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1,bajo,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,bajo,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
3,alto,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,True,False,False,False
4,medio-bajo,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,medio-alto,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
692496,bajo,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
692497,medio-bajo,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,True,False,False
692498,bajo,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False


In [ ]:

# Separar datos en entrenamiento y prueba
X = data_c.drop(['RENDIMIENTO_GLOBAL'], axis=1)  # Predictor variables
y = data_c['RENDIMIENTO_GLOBAL']  # Target variable



In [ ]:
X

,ESTU_PRGM_ACADEMICO_TOP_10_BCTRLG Y LBRTR CLNC,ESTU_PRGM_ACADEMICO_TOP_10_BLG,ESTU_PRGM_ACADEMICO_TOP_10_CMNCCN_Y_PRDSM,ESTU_PRGM_ACADEMICO_TOP_10_CMRC NTRNCNL,ESTU_PRGM_ACADEMICO_TOP_10_CNCS MLTRS,ESTU_PRGM_ACADEMICO_TOP_10_CNM,ESTU_PRGM_ACADEMICO_TOP_10_CNTDR,ESTU_PRGM_ACADEMICO_TOP_10_DMNSTRCN,ESTU_PRGM_ACADEMICO_TOP_10_DNTLG,ESTU_PRGM_ACADEMICO_TOP_10_DRCH Y CNCS PLTCS,...,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
3,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
692496,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
692497,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
692498,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False


In [ ]:
y

0         medio-alto
1               bajo
2               bajo
3               alto
4         medio-bajo
             ...    
692495    medio-alto
692496          bajo
692497    medio-bajo
692498          bajo
692499          alto
Name: RENDIMIENTO_GLOBAL, Length: 692500, dtype: object

In [ ]:
# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((623250, 148), (69250, 148), (623250,), (69250,))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
class_weights = {'alto': 1, 'bajo': 1, 'medio-alto': 3, 'medio-bajo': 3}

In [ ]:
# Modelo 1
rf1 = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=class_weights)
rf1.fit(X_train, y_train)
y_pred1 = rf1.predict(X_test)
print("Modelo 1")
print(classification_report(y_test, y_pred1))
print("Accuracy:", accuracy_score(y_test, y_pred1))


Modelo 1
              precision    recall  f1-score   support

        alto       0.52      0.44      0.48     17645
        bajo       0.41      0.30      0.35     17174
  medio-alto       0.29      0.33      0.31     17198
  medio-bajo       0.30      0.38      0.33     17233

    accuracy                           0.37     69250
   macro avg       0.38      0.36      0.37     69250
weighted avg       0.38      0.37      0.37     69250

Accuracy: 0.36505415162454874


In [ ]:

# Modelo 2
rf2 = RandomForestClassifier(n_estimators=200, random_state=42, max_depth=10, class_weight=class_weights)
rf2.fit(X_train, y_train)
y_pred2 = rf2.predict(X_test)
print("Modelo 2")
print(classification_report(y_test, y_pred2))
print("Accuracy:", accuracy_score(y_test, y_pred2))


In [ ]:

# Modelo 3
rf3 = RandomForestClassifier(n_estimators=100, random_state=42, max_features='sqrt',  max_depth=20, class_weight=class_weights)
rf3.fit(X_train, y_train)
y_pred3 = rf3.predict(X_test)
print("Modelo 3")
print(classification_report(y_test, y_pred3))
print("Accuracy:", accuracy_score(y_test, y_pred3))


Envio de archivo final

In [ ]:
data_test = clean_data(pd.read_csv('test.csv'))

In [ ]:
data_test

,ESTU_PRGM_ACADEMICO_TOP_10_BCTRLG Y LBRTR CLNC,ESTU_PRGM_ACADEMICO_TOP_10_BLG,ESTU_PRGM_ACADEMICO_TOP_10_CMNCCN_Y_PRDSM,ESTU_PRGM_ACADEMICO_TOP_10_CMRC NTRNCNL,ESTU_PRGM_ACADEMICO_TOP_10_CNCS MLTRS,ESTU_PRGM_ACADEMICO_TOP_10_CNM,ESTU_PRGM_ACADEMICO_TOP_10_CNTDR,ESTU_PRGM_ACADEMICO_TOP_10_DMNSTRCN,ESTU_PRGM_ACADEMICO_TOP_10_DNTLG,ESTU_PRGM_ACADEMICO_TOP_10_DRCH Y CNCS PLTCS,...,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
1,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
3,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296781,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
296782,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
296783,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
296784,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
modeloFinal = rf3

In [ ]:
predictions = modeloFinal.predict(data_test)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- ESTU_PRGM_ACADEMICO_TOP_10_FSC
- ESTU_PRGM_ACADEMICO_TOP_10_GLG
- ESTU_PRGM_ACADEMICO_TOP_10_GRNM
- ESTU_PRGM_ACADEMICO_TOP_10_HSTR
- ESTU_PRGM_ACADEMICO_TOP_10_PTMTR
- ...
Feature names seen at fit time, yet now missing:
- ESTU_PRGM_ACADEMICO_TOP_10_CMRC XTRR
- ESTU_PRGM_ACADEMICO_TOP_10_CNC D L N Y BBLTCLG
- ESTU_PRGM_ACADEMICO_TOP_10_DPRT
- ESTU_PRGM_ACADEMICO_TOP_10_HTLR Y TRSM
- ESTU_PRGM_ACADEMICO_TOP_10_N NTRNMNT DPRTV
- ...


In [ ]:
predictions

array(['medio-alto', 'medio-bajo', 'medio-alto', ..., 'medio-alto',
       'medio-alto', 'medio-alto'], dtype=object)

In [ ]:
predictions.shape

(296786,)

In [ ]:

data_testf = pd.read_csv('test.csv')

In [ ]:
submission = pd.DataFrame({
    'ID': data_testf['ID'],
    'RENDIMIENTO_GLOBAL': predictions
})

In [ ]:
submission.to_csv('submission_rf.csv', index=False)